In [17]:
from web3 import Web3
import solcx
import json

install_solc(version='latest')

s3 = Web3(Web3.HTTPProvider('http://192.168.1.100:8545'))
s3.is_connected()

True

In [18]:
# compilazione del contratto da fle invece che da sorgente, con output ABI e bytecode
compiled_file = solcx.compile_files(
     ["greeter.sol"],
     output_values=["abi", "bin", "bin-runtime"],
     solc_version='0.8.0'
)

In [19]:
contract_id, contract_interface = compiled_file.popitem()

#salvataggio di ABI e bytecode in variabili
bytecode = contract_interface['bin']
abi = contract_interface['abi']

# scrittura ABI su JSON
with open('greeter_abi.json', 'w') as f:
    json.dump(abi, f)

In [20]:
# settaggio account di default
s3.eth.default_account = s3.eth.accounts[0]

# recupero (OPZIONALE) di ABI da file JSON
with open('greeter_abi.json') as f:
    abi = json.load(f)

# creazione del contratto
Greeter = s3.eth.contract(abi=abi, bytecode=bytecode)


In [21]:
# deploy del contratto e invocazione del costruttore
tx_hash = Greeter.constructor().transact()

# attesa della conferma della transazione
tx_receipt = s3.eth.wait_for_transaction_receipt(tx_hash)

# creazione di un'istanza del contratto con assegnazione di abi e indirizzo del contratto
greeter = s3.eth.contract(
     address=tx_receipt.contractAddress,
     abi=abi
)

In [22]:
# invocazione della funzione greet() del contratto, 
# che restituisce il valore di default della variabile impostata nel costruttore del contratto 
greeter.functions.greet().call()

'Hello'

In [23]:
# invocazione della funzione setGreeting() del contratto,
# che modifica il valore della variabile greeting ad uno a scelta dell'utente
tx_hash = greeter.functions.setGreeting('Ciao, come stai?').transact()

# attesa della conferma della transazione
tx_receipt = s3.eth.wait_for_transaction_receipt(tx_hash)

# esempio di invocazione della funzione greet() del contratto,
# che restituisce il valore della variabile greeting modificato
greeter.functions.greet().call()

'Ciao, come stai?'